In [1]:
import numpy as np
import tables as tb
import pandas as pd
import glob
import cProfile
import os

El estudio lo vamos a hacer sobre los datos de los años 2012 al 2015, ambos incluidos.
Obtenemos un listado de los ficheros que vamos a cargar en un dataframe.

In [2]:
path="./precip/"
lstFiles = []
lstDir = os.walk(path)
for root, dirs, files in lstDir:
    for fichero in files:
        (nombreFichero, extension) = os.path.splitext(fichero)
        if(extension == ".txt"):
            lstFiles.append(nombreFichero+extension)

Una vez obtenido el listado, leemos los ficheros para obtener un csv

In [3]:
precip_completo = pd.DataFrame()
temp = pd.DataFrame()
aux = []
for lsfile in lstFiles:
    temp = pd.read_csv(path + lsfile,index_col=None, header=0, converters={'YearMonthDay':str, 'WBAN':str})
    aux.append(temp)

precip_completo = pd.concat(aux)
precip_completo.head()

,WBAN,YearMonthDay,Tmax,TmaxFlag,Tmin,TminFlag,Tavg,TavgFlag,Depart,DepartFlag,...,AvgSpeed,AvgSpeedFlag,Max5Speed,Max5SpeedFlag,Max5Dir,Max5DirFlag,Max2Speed,Max2SpeedFlag,Max2Dir,Max2DirFlag
0,03013,20120101,36,,10,,23,,M,,...,5.9,,17,,360,,14,,200,
1,03013,20120102,32,,6,,19,,M,,...,6.4,,18,,260,,14,,200,
2,03013,20120103,43,,12,,28,,M,,...,8.2,,17,,230,,16,,230,
3,03013,20120104,38,,17,,28,,M,,...,7.4,,16,,210,,15,,210,
4,03013,20120105,39,,20,,30,,M,,...,7.4,,18,,210,,17,,210,


Volvamos todos los datos a un csv para comprobar posteriormente su tamaño y poder coparar los tiempos

In [4]:
%time precip_completo.to_csv('precip_completo.csv')

Wall time: 14.5 s


En el csv tenemos demasiados datos que no vamos a utilizar, por lo que quitamos las columnas innecesarias. Tambien creamos una columna nueva con el año

In [5]:
%time precip=precip_completo[['YearMonthDay',"PrecipTotal"]].copy()
%time aux = precip.YearMonthDay.str[:4]
%time precip['Year'] = aux
precip.head(10)

Wall time: 96 ms
Wall time: 981 ms
Wall time: 16 ms


,YearMonthDay,PrecipTotal,Year
0,20120101,0.00,2012
1,20120102,0.00,2012
2,20120103,0.00,2012
3,20120104,0.00,2012
4,20120105,0.00,2012
5,20120106,0.00,2012
6,20120107,0.00,2012
7,20120108,0.00,2012
8,20120109,0.00,2012
9,20120110,0.00,2012


Limpiamos los datos, ya que hay valores que no son numéricos en la columna PrecipTotal, que sí debería serlo. Estos valores erroneos los suponemos 0

In [6]:
%time precip.PrecipTotal.replace(['M','  T','err'], ['0.00','0.00','0.00'], inplace=True)
%time precip['PrecipTotal'] = precip['PrecipTotal'].astype('float') 

Wall time: 418 ms
Wall time: 285 ms


Ahora ya podemo trabajar con los datos.
Vamos a calcular el día de máxima precipitación

In [7]:
%time precip[precip['PrecipTotal']==max(precip['PrecipTotal'])]

Wall time: 282 ms


,YearMonthDay,PrecipTotal,Year
3256,20150702,28.02,2015


In [8]:
%time grouped = precip.groupby('Year').mean()
grouped

Wall time: 210 ms


,PrecipTotal
Year,
2012,0.077193
2013,0.086452
2014,0.086585
2015,0.075724


Ahora vamos a hacer los mismo guardando los datos en un archivo hdf5
vamos a cargar los datos en un dataframe por cada año

In [11]:
%time aux = precip.YearMonthDay.str[:4]
%time precip_completo['Year'] = aux
%time h5file = pd.HDFStore("precip_completo.h5")
%time precip2012 = precip_completo[precip_completo['Year']=='2012']
%time precip2013 = precip_completo[precip_completo['Year']=='2013']
%time precip2014 = precip_completo[precip_completo['Year']=='2014']
%time precip2015 = precip_completo[precip_completo['Year']=='2015']
%time precip2012 = precip_completo[precip_completo['Year']=='2012']
%time precip2013 = precip_completo[precip_completo['Year']=='2013']
%time precip2014 = precip_completo[precip_completo['Year']=='2014']
%time precip2015 = precip_completo[precip_completo['Year']=='2015']
%time h5file['hdf_2012'] = precip2012
%time h5file['hdf_2013'] = precip2013
%time h5file['hdf_2014'] = precip2014
%time h5file['hdf_2015'] = precip2015

Wall time: 620 ms
Wall time: 53 ms
Wall time: 7 ms
Wall time: 441 ms
Wall time: 464 ms
Wall time: 470 ms
Wall time: 556 ms
Wall time: 437 ms
Wall time: 444 ms
Wall time: 439 ms
Wall time: 524 ms
Wall time: 6.26 s
Wall time: 6.48 s
Wall time: 6.03 s
Wall time: 8.13 s


In [12]:
h5file.close()

Vamos a ver qué fichero ocupa más

In [15]:
%ls -l precip_completo.*

 El volumen de la unidad C es Windows
 El n£mero de serie del volumen es: 8C9B-8A73

 Directorio de C:\Users\Sergio_2\Desktop\master\Entornos data science en Python\proyecto fin modulo\entrega 2


 Directorio de C:\Users\Sergio_2\Desktop\master\Entornos data science en Python\proyecto fin modulo\entrega 2

25/10/2016  21:27       276.359.308 precip_completo.csv
25/10/2016  21:30       421.401.237 precip_completo.h5
               2 archivos    697.760.545 bytes
               0 dirs  193.016.160.256 bytes libres


Vemos que el CSV ocupa menos espacio en el disco (en teoría debería ser al contrario)

El resto del ejercicio no he podido hacerlo con HDF5